In [4]:
import pandas as pd, numpy as np, random
import imblearn
random.seed(0)
OVERSAMPLE = False
AUGMENT = False
AUGMENT_NEOPLASIA = True

# Augmentation

Augment dataset by generating new neoplasia samples by averaging random neoplasia samples. This is better than scaling records (think of the effect on the spectra) imo - but this can be tested!

I am augmenting using the training set, to avoid information leakage

I am only uagmenting neoplasia, since looking at evaluation metrics and the spectra, squamous tissue is already successfuly / easily identified.

In [2]:
data = pd.read_csv('original_data/noExclusion_train_data.csv', header=None).reset_index(drop=True)
label = pd.read_csv('original_data/noExclusion_train_label.csv', header=None, names=['label']).astype(int)
combined = pd.merge(data, label, left_index=True, right_index=True)

# print(combined['label'].value_counts())
# 1: 137, 2: 257, 3: 178
squamous = combined.loc[combined['label']==1].reset_index(drop=True)
ndbe = combined.loc[combined['label']==2].reset_index(drop=True)
neoplasia = combined.loc[combined['label']==3].reset_index(drop=True)

In [3]:
# augment dataset (balance NDBE and neoplasia)

if OVERSAMPLE:
    # need to make 79 new neoplasia records
    for i in range(79):
        rand1 = random.randint(0,178) # choose any of the neoplasia records
        rand2 = random.randint(0,178)
        records = neoplasia.iloc[[rand1,rand2]]
        # average the 2 records to make a new neoplasia record
        new_neoplasia = records.mean().to_frame().transpose()
        neoplasia = pd.concat([neoplasia, new_neoplasia], axis=0)

    neoplasia.reset_index(drop=True)
    neoplasia['label'] = neoplasia['label'].astype(int)

    augmented = pd.concat([squamous, ndbe, neoplasia]).reset_index(drop=True)

    x_train = augmented.drop('label', axis=1)
    y_train = augmented['label']

    x_train.to_csv('augmented_data/train_data.csv', index=False, header=False)
    y_train.to_csv('augmented_data/train_label.csv', index=False, header=False)

In [4]:
# augment dataset squamous to 150, NDBE and neoplasia to 300
# why? To create more training, whilst not having too much fake data
# any more tha 300 and neoplasia wouldve had more fake data than real
# 300 for NDBE to match neoplasia and balance it, since they have similar spectra
# 150 to squamous so it isn't too underrepresented - although it doesn't really struggle
# since it is so distinct from NDBE and neoplasia

if AUGMENT:
    # need to make 122 new neoplasia records
    for i in range(122):
        rand1 = random.randint(0,178) # choose any of the neoplasia records
        rand2 = random.randint(0,178)
        records = neoplasia.iloc[[rand1,rand2]]
        # average the 2 records to make a new neoplasia record
        new_neoplasia = records.mean().to_frame().transpose()
        neoplasia = pd.concat([neoplasia, new_neoplasia], axis=0).reset_index(drop=True)

    # need to make 43 new ndbe records
    for i in range(43):
        rand1 = random.randint(0,257) # choose any of the ndbe records
        rand2 = random.randint(0,257)
        records = ndbe.iloc[[rand1,rand2]]
        # average the 2 records to make a new ndbe record
        new_ndbe = records.mean().to_frame().transpose()
        ndbe = pd.concat([ndbe, new_ndbe], axis=0).reset_index(drop=True)

    # need to make 13 new squamous records
    for i in range(13):
        rand1 = random.randint(0,137) # choose any of the ndbe records
        rand2 = random.randint(0,137)
        records = squamous.iloc[[rand1,rand2]]
        # average the 2 records to make a new ndbe record
        new_squamous = records.mean().to_frame().transpose()
        squamous = pd.concat([squamous, new_squamous], axis=0).reset_index(drop=True)

    augmented = pd.concat([squamous, ndbe, neoplasia]).reset_index(drop=True)

    x_train = augmented.drop('label', axis=1)
    y_train = augmented['label'].astype(int)

    x_train.to_csv('augmented_datav2/train_data.csv', index=False, header=False)
    y_train.to_csv('augmented_datav2/train_label.csv', index=False, header=False)

In [5]:
# augment dataset (neoplasia only) and create an equal number of augmented data as
# real data - see if it improves recall

if AUGMENT_NEOPLASIA:
    # need to make 178 new neoplasia records
    for i in range(178):
        rand1 = random.randint(0,178) # choose any of the neoplasia records
        rand2 = random.randint(0,178)
        records = neoplasia.iloc[[rand1,rand2]]
        # average the 2 records to make a new neoplasia record
        new_neoplasia = records.mean().to_frame().transpose()
        neoplasia = pd.concat([neoplasia, new_neoplasia], axis=0).reset_index(drop=True)

    augmented = pd.concat([squamous, ndbe, neoplasia]).reset_index(drop=True)

    x_train = augmented.drop('label', axis=1)
    y_train = augmented['label'].astype(int)

    x_train.to_csv('augmented_datav3/train_data.csv', index=False, header=False)
    y_train.to_csv('augmented_datav3/train_label.csv', index=False, header=False)

augmented['label'].value_counts()

0          1          2          3          4          5          6          7          8          9          10         11         12         13        14         15         16         17         18        19        20        21        22        23        24        25        26        27        28        29        30        31        32        33        34        35       36        37        38        39        40        41        42        43        44       45        46        47        48        49        50        51        52        53        54        55        56        57        58        59        60        61        62        63       64        65        66        67        68        69       70        71        72       73        74        75        76        77       78       79        80       81       82       83       84       85        86        87       88        89        90       91       92       93       94       95        96       97       98        99        100

In [11]:
len(squamous), len(ndbe), len(neoplasia), len(augmented)
augmented['label'].value_counts()
len(x_train), len(y_train)
y_train
y_train.value_counts()

3    356
2    257
1    137
Name: label, dtype: int64

# Data generation